<a href="https://colab.research.google.com/github/microprediction/endersnotebooks/blob/main/mean_reversion_attacker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade git+https://github.com/microprediction/endersgame.git

  Cloning https://github.com/microprediction/endersgame.git to /tmp/pip-req-build-aibp23p1
  Running command git clone --filter=blob:none --quiet https://github.com/microprediction/endersgame.git /tmp/pip-req-build-aibp23p1
  Resolved https://github.com/microprediction/endersgame.git to commit f7cd9a25b7de1af86c92cc6a1beaab736f92db1a
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.6 MB/s eta 0:00:00
  Created wheel for endersgame: filename=endersgame-0.0.9-py3-none-any.whl size=19821 sha256=87058d95a267cfab894c05afa2fab1ce1cfef657c28dbe0574512dcbf021714a
  Stored in directory: /tmp/pip-ephem-wheel-cache-wzn1vvqs/wheels/39/24/f0/19aeef5765f9b9f629bab092893ebd3c04bde902d978c742bb
Successfully built endersgame


# Creating and testing an attacker
You will participate in "Enders Game" ... a sport that is more serious than you can possibly imagine.

- This notebook demonstrates how to create an "attacker" (see [README.md](https://github.com/microprediction/endersgame/tree/main/endersgame/attackers)), and test it.

- We will use data endersgame steam generator (see this [notebook](https://github.com/microprediction/endersnotebooks/blob/main/enders_data_generator.ipynb)).

## What should an attacker do?

In short, it tries to predict `up` or `down` but not too often.

Our attacker will consume a univariate sequence of numerical data points $x_1, x_2, \dots x_t$ and look for occasional deviations from the [martingale property](https://en.wikipedia.org/wiki/Martingale_(probability_theory), which is to say that we expect the series $x_t$ to satisfy:

$$ E[x_{t+k}] \approx x_t $$

The attacker signals whether the future value will, on average, be above or below the current value.

## Overview
We will


1.   Start with an attacker that already has some accounting logic
2.   Modify the default `tick` and `predict` methods
3.   Run the attacker on mock data
4.   Run the attacker on real data

Afterwards we will

4.   Set up an optimization to tune the attacker's parameters
5.   See if it helps on the test set


## Imports


In [98]:
from endersgame.attackers.attackerwithsimplepnl import AttackerWithSimplePnL
from endersgame.rivertransformers.macd import MACD
from endersgame.datasources.streamgenerator import stream_generator
from river import stats
import numpy as np
import math
import types
from pprint import pprint

## Step 1: Decide what state to maintain
Let's first implement the `tick` method. This should quickly respond to an incoming data point. Here we choose to maintain the current value and also an exponentially weighted moving average of historical values.

In [99]:
from endersgame.attackers.attackerwithsimplepnl import AttackerWithSimplePnL

class MyAttacker(AttackerWithSimplePnL):

     def __init__(self, a=0.01):
        super().__init__()
        self.state = {'running_avg':None,
                      'current_value':None}
        self.params = {'a':a}

     def tick(self, x:float):
         # Maintains an expon moving average of the data
         self.state['current_value'] = x
         if not np.isnan(x):
            if self.state['running_avg'] is None:
                self.state['running_avg'] = x
            else:
                self.state['running_avg'] = (1-self.params['a'])*self.state['running_avg'] + self.params['a']*x


### Testing tick
Instantiate the attacker and let it tick on data from history.

In [100]:
x_train_stream = stream_generator(stream_id=0,category='train')
attacker = MyAttacker()
for x in x_train_stream:
    attacker.tick(x)

last_x = x
print(f"After processing the entire stream, the current value is  {attacker.state['current_value']} and the moving average is {attacker.state['running_avg']}")

No more files found for stream_id=0 in category='train'.
After processing the entire stream, the current value is  9583.964285712302 and the moving average is 9583.164896526241


## Making an `up` or `down` decision
Next we implement `predict` using a mean reversion strategy.

In [101]:
def predict(self, k:int=None)->float:
    if self.state['current_value'] > self.state['running_avg'] + 1:
        return -1
    if self.state['current_value'] < self.state['running_avg'] - 1:
        return 1
    return 0

attacker = MyAttacker()
attacker.predict = types.MethodType(predict, attacker) # <-- Attach the predict method to our existing instance of attacker


Let's check that if the current value is very high we should predict it will fall:

In [103]:
attacker.state['current_value'] = 10
attacker.state['running_avg'] = 5
print(attacker.predict())

-1


## Run the attacker on mock data
We'll attack the prediction method to the attacker then run it

In [105]:
k=100                           # Prediction horizon
attacker = MyAttacker()         # Always reset an attacker
attacker.predict = types.MethodType(predict, attacker)

s = [1,3,4,2,4,5,1,5,2,5,10]*100
for x in xs:
   y = attacker.tick_and_predict(x=x, k=k)

## Run the attacker on real data

In [106]:
k = 100       # Horizon
x_test_stream = stream_generator(stream_id=1,category='train')
attacker = MyAttacker()
attacker.predict = types.MethodType(predict, attacker)     #  <-- If you get sick of doing this then put the method in the class at the outset
for x in x_test_stream:
    y = attacker.tick_and_predict(x=x,k=k)

attacker.state

No more files found for stream_id=1 in category='train'.


{'running_avg': 6415.837463556068, 'current_value': 6415.239999998541}

## Check the attacker's profit and loss


In [108]:
pprint(attacker.get_pnl_summary())

{'current_ndx': 46480,
 'losses': 96,
 'num_resolved_decisions': 203,
 'profit_per_decision': 0.012241379310219474,
 'standardized_profit_per_decision': 0.0098951723910636,
 'total_profit': 2.484999999974553,
 'win_loss_ratio': 1.1145833333333333,
 'wins': 107}
